# How do Climate Fluctuations Affect Crop Yields?


## Pathway

This report maps out the steps we took to find correlations in weather patterns and crop yields for oranges, grapefruits, and lemons in California. The report is divided into the following sections:

* Developing a Project Focus
* Data Acquisition
* Data Cleaning and Transformation
* Identifying Questions
* Methods | Building Models
* Analyzing Data through Correlation Matrices
* Validating and Evaluating the Analysis and Models
* Challenges | New Questions
* Lessons Learned
* Future Studies
* Resources

# Developing a Project Focus : Starting with Smart Yields and Moving Toward Citrus




### Weather Anomalies Over Time
We started analyzing the Smart Yields dataset with an initial interest in analyzing the effect weather anomalies have on crop yields over time after learning about the work of Vincent Kimura and Axel Holland for their startup. In an attempt to help them build stronger models, we wanted to provide insights into what variables may be important indicators in identifying impending frost. However, we realized that after plotting the Smart Yields time series, the data only spanned from March 27th of this year until May 8th. We needed to augment the data to develop a more robust dataset, which would give us an opportunity to use the techniques we learned in CIS 705. 

Smart Yields included data on apples. Fruit trees are particularly sensitive to fluctuations in weather, so we decided that choosing fruit tree crops, in particular, might show a relatively high amount of variation and make for an interesting project. We knew we wanted to analyze the impact of weather on fruit trees over time.

### The appeal of citrus
For ease of modeling, we simplified it one step further and selected citrus. The three broad categories of citrus crops that we decided to focus on were oranges, grapefruit, and lemons. This choice was was primarily due to the data that was available to us.

We decided that citrus trees would make for an interesting study crop, as citrus trees are susceptible to extremes in temperature, since because they do not hibernate or shed leaves in winter. Also, [per capita consumption of oranges is higher than that any other fruit](https://www.agmrc.org/commodities-products/fruits/citrus/citrus-profile/). Finally, the citrus industry has large economic contributions. For instance, it adds an [annual seven billion dollars to the California economy](https://www.westernfarmpress.com/fruit/citrus-pumps-71-billion-through-californias-economy) alone.

### Selecting California
Due to this economic impact in California, as well as the variety and volume of citrus produced in the state, we decided to focus on California as a case study for our exploration of the effects of weather on citrus production. California is also one of the top four citrus producing states in the nation, alongside Texas, Arizona, and Florida. 

The following pie charts break down the production by percentage for oranges, grapefruit, and lemons by state:

<div style="display: flex;">
<img style= "float: left;" src="./images/lemonspie.jpg" height="400" width="400"> 
<img style= "float: center;" src="./images/grapefruitpie.jpg" height="400" width="400"> 
<img style= "float: right;" src="./images/orangespie.jpg" height="400" width="400">
</div>



# Data Acquisition

## The Agricultural Data

<img src="./images/Economic_Research_Service_Logo.PNG" height=400 width=400>

The agricultural yield and bearing acreage time series data was acquired from the United States Department of Agriculture's [Economic Research Service.](https://www.ers.usda.gov/data-products/fruit-and-tree-nut-data/fruit-and-tree-nut-yearbook-tables/) The USDA Fruit and Tree Nut Yearbook tables contain almost four decades of time-series data for U.S. bearing acreage, production, prices, etc. for a variety of Fruit and Nut Tree crops. We acquired our citrus crop data from a collection of these Fruit Tree Yearbook tables that look at bearing acreage, and yield per acre, for a time series ranging from 1980 through 2018. The citrus crop tables contained aggregated data for lemons, oranges, and grapefruit, for the states of California, Texas, Arizona, and Florida.

## The Weather Data

<img src="./images/National_Climatic_Data_Center_logo.png">

The weather timeseries data was acquired from the NOAA's National Climatic Data Center, or NCDC for short. [An article on citrus production in California (page 2)](https://apps1.cdfa.ca.gov/FertilizerResearch/docs/Citrus_Production_CA.pdf) revealed that the counties of Kern, Fresno, Tulare, Ventura, and Riverside as being the top producers of citrus in the state of California, we decided to pull weather readings from stations in each of these five counties. The [Local Climatological Data](https://www.ncdc.noaa.gov/cdo-web/datatools/lcd) tool at the NCDC's website allowed us to pull hourly station readings, of temperature and relative humidity, from each of these five counties, spanning a time series from 2000 - 2018. The data came in the form of .csv files, which made them easy to process, but there was a significant delay between requesting the data, and when the data was actually served to us. Of that weather data, we decided to focus only on readings of hourly dry bulb temperature, and hourly relative humidity.

# Data Cleaning and Transformation

While the agricultural data gathered from the USDA was in .csv files, the organization of the data within these files was awkward, which necessitated the use of a custom python script to convert the data into rows that could easily be processed and analyzed.

While the agricultural data came in the form of a yearly time series of aggregate yield per acre and bearing acreage, the weather data consisted of raw station readings recorded at hourly intervals. Therefore, in order to match the weather data with the agricultural data on the same temporal scale, we needed to aggregate the hourly weather station readings into daily, monthly, and yearly summary statistics. We generated the following summary statistics for temperature and humidity.

- The average, min, and max temperature and relative humidity readings for each year.
- Temperature range (max - min)
- Number of days with temperatures 10 degrees above the yearly average, by year
- Number of days with temperatures 10 degrees below the yearly average, by year
- Number of days with minimum temperatures below 32 degrees, by year
- Number of days with minimum temperatures within 3 degrees of the monthly minimum, in months with extreme lows, by year
- Number of days with temperatures 7 degrees above the monthly average, by year
- Number of days with temperatures 7 degrees below the monthly average, by year
- Number of days with relative humidity 15% or more above the monthly average, by year
- Number of days with relative humidity 25% or more above the yearly average

These summary statistics were generated using a combination of SQL queries to generate the original yearly, monthly, and daily aggregates, and python list comprehensions to take care of the more complicated metrics.

After having generated these yearly aggregate statistics for weather, we then joined the agricultural data with the weather data into pandas DataFrames objects. Working with DataFrames gave us easy to use snapshots of our data, and made generating plots and correlation statistics of the various columns both easy and intuitive.

![Pandas DataFrame](./images/pd_data_frame.png)

*Example DataFrame generated for lemons*

# Identifying Questions

## Noticing Trends in Texas | How are crops affected by freezing temperatures?
We first noticed a delay between freeze event occurrences and reported bearing acreage in Texas in 1984. This pattern was seen once again with grapefruit and oranges between 1989 and 1990. Therefore, for analyzing the causative factors behind the trends of bearing acreage over time, it seemed important to look at extreme temperature lows that occurred during the previous season. 

An article from [the Texas Agricultural Extension Service](https://aggie-horticulture.tamu.edu/citrus/l2319.htm) mirrored our findings, further highlighting the severity of crop loss experienced a succeeding season after the frosts.

![Texas Timeseries](./images/texas.jpg)


# Methods | Building Models

## Finding an Initial Model
During our literature review, we happened upon [a 2010 article](https://core.ac.uk/download/pdf/6550507.pdf) from the Agricultural and Applied Economics Association that suggests that bearing acreage and yield for fresh grapefruit and oranges are partially dependent upon climate. 

In particular, the article presents the following models for bearing acreage and yield for fresh oranges and grapefruit, which we have summarized below.

$Yield_t = f(Climate_t, Prices_{t - 1}, Technology_t, LandQuality_t)$

$Acreage_t = f(Acreage_{t - 1}, Acreage_{t - 2}, Climate_{t - 1}, Prices_{t - 1}, ...)$

Without focusing too much on the details, we noticed that both of these models include climate as an independent variable. We also noted that while the yield for a crop at timestep $t$ is dependent upon the climatic conditions at that same time step, the acreage is dependent upon the conditions at timestep $t - 1$, indicating a lag in the effects of climate on the bearing acreage of citrus crops.

## Linear Regression
For our modeling, we wanted to start with a big picture overview before deep diving into the granular elements, using scatterplots and linear regression of citrus bearing acreage with aggregated totals in the United States to start initital analysis.

*Practical Statistics for Data Scientists* emphasizes “by itself, the regression equation does not prove the direction of causation. Conclusions about causation must come from a broader context of understanding about the relationship.” For instance, nationally, there has been a noticeable decrease in bearing acreage for oranges, grapefruit, and lemons. in the past twenty years with a predicted decrease in the next five years. This answers the “what,” but further exploration of correlation is needed to discover the possibilities in the “why.”

<img src="./images/bearingacreage.png">



# Analyzing Data through Correlation Matrices

## Production | Bearing Acreage | Yield Per Acre
By merging weather data with yield data, we were able to analyze correlations between the yearly summary statistics for weather and citrus crop data. 

### Production vs. Freezing Temperatures
We observed a strong negative correlation between the number of days out of the year where the temperature was below freezing and the production of grapefruit (-0.74). The same effect to a lesser degree was seen in lemons (-0.27) and oranges (-0.02).

<div style="display: flex;">
<img style= "float: left;" src="./images/g_bf_p_snc_.jpg" height="600" width="600"> 
<img style= "float: right;" src="./images/l_o_bf_p_mwnc.jpg" height="600" width="600">
</div>

### Bearing Acreage vs. Below Freezing Temperatures
There is a moderate negative correlation between days with temperatures below freezing and bearing acreage for all three citrus crops (grapefruit, -0.55; oranges, -0.31; lemons, -0.27).

<img src="./images/o_g_l_bf_ba_mnc.jpg">

### Bearing Acreage vs. Temperature Range                             
When looking at temperature range versus bearing acreage within a year, we observed a strong negative correlation in oranges (-0.72), as well as a moderate negative correlation in grapefruit (-0.53) and lemons (-0.42).

<div style="display: flex;">
<img style="float: left;" src="./images/o_tr_ba_snc.jpg" height="600" width="600">
<img style="float: right;" src="./images/g_l_tr_ba_mnc.jpg" height="600" width="600">
</div>

### Bearing Acreage vs. Minimum Temperature
When looking at minimum temperature versus bearing acreage in a given year, we observed a strong positive correlation in oranges (0.73) and a moderate positive correlation in grapefruit (0.48) and lemons (.40). This meant that as the higher the minimum temperature was in a year, the higher the bearing acreage.

<div style="display: flex;">
<img style="float: left;" src="./images/o_mt_ba_spc.jpg" height="600" width="600">
<img style="float: right;" src="./images/g_l_mt_ba_mpc.jpg" height="600" width="600">
</div>

### Yield Per Acre vs. Number of Days with Temperature Below the Yearly Average Temperature
When looking at the number of days with an average temperature below the yearly average temperature within a given year, we found moderate negative correlations with yield per acre for all three citrus crops (grapefruit, -0.36; oranges, -0.29; lemons, -0.44).

<img src="./images/o_g_l_atb_ypa_wnc.jpg">

### Yield Per Acre vs. Number of Days with Temperature Above the Yearly Average Temperature
We observed similar relationships between the number of days with above average temperatures and yield per acre across all three citrus crops (grapefruit, -0.60; oranges, -0.43; lemons, -0.19).

<img src="./images/o_g_l_ata_ypa_wsnc.jpg">

### Bearing Acreage vs. Yield Per Acre
In generating the correlation matrices, we saw a moderate-to-strong negative correlation between bearing acreage and yield per acre across all three citrus crops (grapefruit, -0.68; oranges, -0.55; lemons, -0.38). This led to a new question: Why does the yield per acre decrease as the bearing acreage increases?

<img src="./images/o_g_l_ypa_ba_msnc.jpg">

# Validating and Evaluating the Analysis and Models

## Hypothesis Testing - Pearson's Correlation Statistic

We performed hypothesis testing to try and validate the observed linear relationships between some of the weather and agricultural variables analyzed above. Our null hypotheses in all cases were that there is no linear relationship between each pair of weather and agricultural variables that we looked at. The test statistic that we used to test the null hypotheses was the Pearson's correlation coefficient. Under the assumption that the null hypothesis is true, we used a p-value of 0.05 to determine statistical significance. All p-values were generated via the `scipy.stats` package's built in `pearsonr()` function.

### Days with Below Freezing Temperatures vs. Production

When looking at the linear relationship between the number of days with below freezing temperatures in a year, and production measured in 1k short tons, we observed a strong negative correlation in grapefruit, a moderate negative correlation in lemons, and a weak negative correlation in oranges. When looking at the p-values, we see that this linear relationship is only statistically significant in grapefruit, which had a p-value of 0.0004. The p-values for oranges and lemons on the other hand were 0.9266 and 0.2736 respectively, indicating a failure to reject the null hypothesis that there is no linear relationship between these two variables in these crops. One would think that this is indicative of grapefruits being biologically less cold hardy than other citrus varieties, however our literature review has [indicated otherwise.](https://aggie-horticulture.tamu.edu/newsletters/hortupdate/2011/mar/citrus_freeze.html). However, oranges are considered to be somewhat more cold hardy than other fruits, requiring temperatures in the mid-20s before damage is incurred, which aligns with the lack of a strong linear relationship observed between freezing temperatures and production in this corp. Another possibility is that perhaps the orange and lemon farms whose data was collected by the USDA had employed countermeasures that allowed them to mitigate the effects of these extremes in cold on their crops, whereas the grapefruit farms did not. Fine grained data at the county and farm level would be required to validate and test this hypothesis though, which points to a potential direction for future study in this area.

### Days with Below Freezing Temperatures vs. Bearing Acreage

When looking at the potential linear relationship between the number of days with below freezing temperatures, and bearing acreage measured in 1k acres, we had observed moderate negative Pearson's correlations across all three citrus crops. When looking at the p-values, we saw that this relationship is only statistically valid in grapefruits and lemons, with p-values of 0.0186 and 0.0109 respectively. The p-value for oranges was 0.2181, indicating a failure to reject the null hypothesis that there is no linear relationship between these two variables in oranges. As highlighted previously, oranges are considered to be one of the more cold hardy citrus varieties, which might explain the lack of a statistically significant negative linear relationship between freezing temperatures and bearing acreage in this crop.

### Temperature Range vs. Bearing Acreage

When looking at the potential linear relationship between the yearly temperature range (max fahrenheit - min fahrenheit) and bearing acreage, we had observed a strong negative correlation in oranges, and moderate negative correlations in grapefruits and lemons. When looking at the p-values, we see that this relationship is only statistically significant in grapefruits and oranges, which had p-values of 0.0223 and 0.0007. The p-value for lemons was 0.0866, indicating a failure to reject the null hypothesis that there is no linear relationship between this two variables with regards to lemons.

### Minimum Temperature vs. Bearing Acreage

When looking at the potential linear relationship between the minimum yearly temperature in fahrenheit and the bearing acreage, we had observed a strong positive correlation in oranges, and moderate positive correlations in grapefruits and lemons. When looking at the p-values, we again see that this relationship is statistically significant in grapefruits and oranges, with p-values of 0.0425 and 0.0006 respectively, while not statistically significant in lemons, which had a p-value of 0.1032. This is to be expected after having seen similar results for temperature range, as temperature range is just a linear function of minimum temperature. In general, we would expect that as minimum yearly temperature goes up (indicating a less extreme low), the bearing acreage for the citrus crops in the respective season would be higher. The lack of a statistically significant relationship in lemons in conjunction with there being significant results for oranges and grapefruits might simply be indicative of noise in the data, lack of enough data points (the time series we looked at only spans 18 years), or the way that we joined the weather data with the agricultural data. Further exploration and slicing of the data is required to tease apart the complex and multifaceted interactions between the weather and agricultural variables that we analyzed.

### Number of Days with Below Average Temperatures vs Yield Per Acre

When looking at the potential linear relationship between the number of days with average temperatures below the yearly average, and yield per acre, we had observed weak negative correlations between these two variables with respect to all three citrus crops. We also saw that this relationship is not statistically significant in grapefruits, oranges, nor in lemons, which had p-values of 0.1426, 0.2400, and 0.0690 respectively, indicating a failure to reject our null hypothesis across the board.

### Number of Days with Above Average Temperatures vs Yield Per Acre

When looking at the potential linear relationship between yield per acre, and the number of days with average temperatures above the yearly average, we observed weak-to-strong negative correlations in all three citrus crops. When looking at the p-values, we see that this relationship is only statistically significant in grapefruits, which had a p-value of 0.0087. Oranges and lemons on the other hand had p-values of 0.0717 and 0.4615 respectively, indicating a failure to reject the null hypothesis that there is no linear relationship between these two variables with respect to these crops. While we only saw a statistically significant relationship in grapefruits, the existence of these negative relationships indicates that yield per acre in citrus could potentially be negatively effected by extreme highs in temperature, in addition to extreme lows. Such declines in yield per acre might potentially be due to damage caused by dessication or heat stress under high temperature regimes, but more analysis and data collection is needed to validate this hypothesis.


### Bearing Acreage vs. Yield Per Acre

When looking at bearing acreage vs. yield per acre in the three citrus crops, we observed moderate to strong negative correlations between these two variables in all three. However, when looking at the p-values, we see that this relationship is only statistically significant in grapefruit and oranges, with p-values of 0.0018 and 0.0178 respectively. When looking at the relationship between these two variables in lemons, we see a p-value of only 0.1225, indicating a failure to reject the null hypothesis, that there is no linear relationship between these two variables in lemons. The presence of this linear relationship came as a surprise to us, and has led to the development of more hypotheses that we would like to explore in a future study. One hypothesis for this observed negative relationship between bearing acreage and yield per acre, is that perhaps as citrus farms are scaling back their bearing acreage, they are better able to utilize and tend to the producing acreage that they alread have, which is leading to an increase the the measured yield per acre. As the USDA data is aggregated, and does not reveal such fine grained details, more data collection at the county and farm level would be needed to explore and test this hypothesis.

### Issues

One issue that we had with attempting to evaluate the linear relationship between freezing temperatures and production in this study, is that production is a component not only of bearing acreage, but also yield per acre. The weather data that we collected spans the calendar year, whereas the agricultural data, while yearly, actually corresponds to the production season for each crop. Orange and grapefruit season does not start until November in California, and lemon season starts in August. Therefore, to better quantify the effects of temperature anomalies on yield per acre, which is more dependent upon recent fluctuations in weather, we would need to reagreggate the weather data into seasonal aggregates, to potentially see stronger relationships between weather variables and yield per acre.

Another issue with using a statistic like number of days with below freezing temperatures as a proxy for extremes in cold, is that it does not provide any information as to the duration of time during which those freezing temperatures occured within a day. Since we generated this statistic by just counting the number of days that had a minimum temperature below freezing, and did not look at the span of hours within the day that had these extreme lows, we missed out on a lot of important information. Citrus crops will be damaged by extreme lows, but that damage will be more prone to manifesting if those extremes in temperature occur over an extended period of time without a break. We had attempted to capture this information by looking at the number of days in a year with temperatures close to their monthly extreme, in months that had extreme temperatures, but this is also a bad metric in hindsight, because only a few months out of an average year would be expected to have extreme lows (below freezing), and when looking at the relationship between this variable and production, we observed weak correlations across all three crops.


# Challenges | New Questions
We wanted to analyze citrus disease as a potential contributing factor to the observed decline of citrus production over time. Unfortunately, finding fine-grain data related to citrus disease proliferation and distribution proved to be difficult. We attempted to use high humidity as a proxy for disease spread. While we did observe a moderate negative correlation between high humidity trends and bearing acreage across all three citrus crops, we acknowledge that using this proxy is bound to introduce a margin of error. Although there may be issues of statistical or scientific significance with this method, the data is still meaningful and worthy of future exploration.

# Lessons Learned
* One Bite at a Time | Our scope was too large at the start and we lacked focus, which may have prevented us from moving as swiftly as we would have liked from the start. When completing a large-scale project, succinct phases are neccessary for success.
* Ethics and Policy Can be Considered No Matter What Data Is Being Explored | Although we were looking at aggregated data on a large scale without record-level information, we can still think about implications of our findings in a broader social context. For example, if we found zero correlation between climate and crop yields, perhaps we would be supporting those who do not believe in climate change. If phase II of this study explores labor data and we state that migrant labor is absolutely necessary to support agriculture in the United States, it may bring up issues of immigration. 
* Compare Apples to Apples, Not Apples to Oranges | We realized that when working with time series data, it was important to be cognizant of the frequency and time intervals in which data points were collected. The agricultural data we pulled from the USDA was broken down by year whereas the weather data was in the form of hourly sensor readings. This made unifying the two data sets into a single yearly time series quite challenging. Planning time for matching variables is essential in the project planning process.

# Future Studies

## Citrus Greening Disease
By scanning news articles and scientific journals, we discovered that a significant portion of the decline of citrus production across the United States can be attributed to the introduction and subsequent spread of citrus greening disease, which is mediated by an insect known as the Asian Citrus Psyllid. Further exploration of the literature revealed that higher humidity trends are more conducive to the spread of citrus greening disease, vectored by the Asian Citrus Psyllid.

## Labor Data
We also hypothesized that another factor that may contribute to the declining production of citrus over time is the potential decline in available agricultural laborers. Perhaps farms are scaling back their planted acreage in response to a reduced ability to meet labor demands to maintain crops. More research and data acquisition from the Bureau of Labor Statistics may provide further insights into this potential relationship.

# Resources

* Anderson, M. (n.d.). Understand How Cold Temperatures Affect Citrus Trees. Retrieved from https://aggie-horticulture.tamu.edu/newsletters/hortupdate/2011/mar/citrus_freeze.html

* Bruce, Peter; Bruce, Andrew (2017). Practical statistics for data scientists: 50 essential concepts (Kindle Locations
 2714-2716). O'Reilly Media. Kindle Edition. 
 
* Citrus: Agricultural Marketing Resource Center (2015). Retrieved from https://www.agmrc.org/commodities-products/fruits/citrus/citrus-profile

* Fitchette, T. (2018, September 12). Citrus pumps $7.1 billion through California's economy. Retrieved from https://www.westernfarmpress.com/fruit/citrus-pumps-71-billion-through-californias-economy

* Frank, M. (2017, June 06). Can America's Farms Survive the Threat of Deportations? Retrieved from https://www.theatlantic.com/business/archive/2017/06/can-americas-farms-survive-the-threat-of-deportations/529008/

* Geisseler, Daniel (2016). Citrus production in California. Retrieved from https://apps1.cdfa.ca.gov/FertilizerResearch/docs/Citrus_Production_CA.pdf

* Gutierrez, A., & Ponti, L. (2013). Prospective analysis of the geographic distribution and relative abuncance of asian citrus psyllid ....*The Florida Entomologist, 96(4)*, 1375-1391.

* Huang, H., & Khanna, M. (2010). An econometric analysis of U.S. crop yield and cropland acreage: Implications for the impact of climate change. *IDEAS Working Paper Series from RePEc, IDEAS Working Paper Series from RePEc, 2010*

* If America is overrun by low-skilled migrants... (2017, July 27). Retrieved from https://www.economist.com/united-states/2017/07/27/if-america-is-overrun-by-low-skilled-migrants

* USDA sees declining production in major US crops. (n.d.). Retrieved from https://www.agri-pulse.com/articles/8960-usda-sees-declining-production-in-major-us-crops